In [36]:
# imports
import cv2
import numpy as np

from utils.opencv import read_video, write_video, determine_area_to_mask, determine_threshold_colors_for_mask

# global parameters
INPUT_FULL_VIDEO_FILE_PATH = "../data/WSA/01-How to charge Pure Charge&Go AX with Pure Dry&Clean _ Signia Hearing Aids.mp4"
OUTPUT_VIDEO_FILE_PATH = "../data/input/test_video.mp4"
INPUT_IMAGE_FILE_PATH = "../data/input/test_image.png"
INPUT_IMAGE_FILE_WITHOUT_TEXT_PATH = "../data/input/test_image_without_text.png"
INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH = "../data/input/test_image_only_hearing_aids.png"
INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH2 = "../data/input/test_image_only_hearing_aids2.png"
INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH = "../data/input/test_image_only_hearing_aids_no_brand.png"
INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_SKELETON_PATH = "../data/input/test_image_only_hearing_aids_skeleton.png"
MASK_FILE_PATH = "../data/input/mask_hearing_aids.png"
MASK_BRAND_FILE_PATH = "../data/input/mask_hearing_aids_brand.png"
MASK_SMOOTH_FILE_PATH = "../data/input/mask_hearing_aids_smooth.png"

# boolean for skipping parts of the processing
CREATE_TEST_VIDEO = False
CREATE_TEST_IMAGE = False
CREATE_TEST_IMAGE_WITHOUT_TEXT = False
CREATE_TEST_IMAGE_ONLY_HEARING_AIDS = False
CREATE_TEST_IMAGE_ONLY_HEARING_AIDS2 = False
REMOVE_BRAND_NAME = False
CREATE_BRAND_MASK = False
CREATE_MASK = False
SMOOTHEN_MASK = False

In [37]:
# create test video
if CREATE_TEST_VIDEO:
    frames_full_video, fps_full_video = read_video(INPUT_FULL_VIDEO_FILE_PATH)
    write_video(OUTPUT_VIDEO_FILE_PATH, frames_full_video[360:800], fps_full_video)

In [38]:
# create test image
if CREATE_TEST_IMAGE:
    frames, _ = read_video(OUTPUT_VIDEO_FILE_PATH)
    img = frames[0]
    print("--- Writing test image ---")
    print(f"path: {INPUT_IMAGE_FILE_PATH}")
    cv2.imwrite(INPUT_IMAGE_FILE_PATH, img)

In [39]:
if CREATE_TEST_IMAGE_WITHOUT_TEXT:
    img = cv2.imread(INPUT_IMAGE_FILE_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates)

    # inpaint with openCV
    new_img = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)

    # save image
    print("--- Writing test image without text ---")
    print(f"path: {INPUT_IMAGE_FILE_WITHOUT_TEXT_PATH}")
    cv2.imwrite(INPUT_IMAGE_FILE_WITHOUT_TEXT_PATH, new_img)

In [40]:
if CREATE_TEST_IMAGE_ONLY_HEARING_AIDS:
    img = cv2.imread(INPUT_IMAGE_FILE_WITHOUT_TEXT_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates, invert=True)

    # inpaint with openCV
    new_img = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)

    # save image
    print("--- Writing test image with only hearing aids ---")
    print(f"path: {INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH}")
    cv2.imwrite(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH, new_img)

In [41]:
if CREATE_TEST_IMAGE_ONLY_HEARING_AIDS2:
    img = cv2.imread(INPUT_IMAGE_FILE_WITHOUT_TEXT_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates)

    # inpaint with openCV
    new_img = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)

    # save image
    print("--- Writing test image with only hearing aids (2) ---")
    print(f"path: {INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH2}")
    cv2.imwrite(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH2, new_img)

In [42]:
if REMOVE_BRAND_NAME:
    img = cv2.imread(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates)

    # inpaint with openCV
    new_img = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)

    # save image
    print("--- Writing test image with only hearing aids and no brand ---")
    print(f"path: {INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH}")
    cv2.imwrite(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH, new_img)

In [43]:
if CREATE_BRAND_MASK:
    img = cv2.imread(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates)

    # save mask
    print("--- Writing mask of brand ---")
    print(f"path: {MASK_BRAND_FILE_PATH}")
    cv2.imwrite(MASK_BRAND_FILE_PATH, mask)

In [44]:
if CREATE_MASK:
    img = cv2.imread(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH)

    # determine mask
    rectangle_coordinates = determine_area_to_mask(img)
    mask, _, _ = determine_threshold_colors_for_mask(img, rectangle_coordinates)

    # save mask
    print("--- Writing mask ---")
    print(f"path: {MASK_FILE_PATH}")
    cv2.imwrite(MASK_FILE_PATH, mask)

In [45]:
if SMOOTHEN_MASK:
    dilate_factor_close = 25
    dilate_factor_open = 3
    mask = cv2.imread(MASK_FILE_PATH, 0)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((dilate_factor_open, dilate_factor_open), np.uint8), iterations=3)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((dilate_factor_close, dilate_factor_close), np.uint8))
    cv2.imwrite(MASK_SMOOTH_FILE_PATH, mask)

In [64]:
def change_color_of_hearing_aids(image_path, image_only_hearing_aids_no_brang_path, mask_hearing_aids_path, mask_brand_path):
    original_image = cv2.imread(image_path)
    img = cv2.imread(image_only_hearing_aids_no_brang_path)
    hsv = cv2.cvtColor(cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2HSV) # converting into HSV color model (first to gray to filter out weird blocky noise)
    h_base, s_base, v_base = cv2.split(hsv)

    mask = cv2.imread(mask_hearing_aids_path, 0)
    mask_brand = cv2.imread(mask_brand_path, 0)
    inv_mask = cv2.bitwise_not(mask)
    inv_mask_brand = cv2.bitwise_not(mask_brand)

    def doNothing(x):
        pass

    # creating a resizable window named Track Bars
    cv2.namedWindow('Track Bars', cv2.WINDOW_NORMAL)

    # creating track bars for gathering threshold values of red green and blue
    cv2.createTrackbar('h', 'Track Bars', 0, 180, doNothing)
    cv2.createTrackbar('s', 'Track Bars', 0, 255, doNothing)
    cv2.createTrackbar('v', 'Track Bars', 0, 255, doNothing)

    # creating a loop to get the feedback of the changes in trackbars
    while True:
        try:
            # reading the trackbar values for thresholds
            h_value = cv2.getTrackbarPos('h', 'Track Bars')
            s_value = cv2.getTrackbarPos('s', 'Track Bars')
            v_value = cv2.getTrackbarPos('v', 'Track Bars')
        except:
            break

        h = np.mod(h_base + h_value, 180)
        s = np.clip(s_base - s_value, 0, 255)
        v = np.clip(v_base + v_value, 0, 255)
        hsv = cv2.merge([h, s, v])

        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        result_raw = cv2.bitwise_or(cv2.bitwise_and(img, img, mask=inv_mask), cv2.bitwise_and(bgr, bgr, mask=mask))
        result_raw_with_brand = cv2.bitwise_or(cv2.bitwise_and(result_raw, result_raw, mask=inv_mask_brand), cv2.bitwise_and(original_image, original_image, mask=mask_brand))
        result = cv2.bitwise_or(cv2.bitwise_and(original_image, original_image, mask=inv_mask), cv2.bitwise_and(result_raw_with_brand, result_raw_with_brand, mask=mask))
        
        # showing the resulting image
        cv2.imshow('Result', result)

        # checking if q key is pressed to break out of loop
        key = cv2.waitKey(25)
        if key == ord('q'):
            break

    # destroying all windows
    cv2.destroyAllWindows()

In [65]:
change_color_of_hearing_aids(INPUT_IMAGE_FILE_PATH, INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH, MASK_SMOOTH_FILE_PATH, MASK_BRAND_FILE_PATH)

In [ ]:
# accidentally came across a black and white see through image with XOR. Could be useful?
img = cv2.imread(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_NO_BRAND_PATH)
original_image = cv2.imread(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_PATH)
mask_brand = cv2.imread(MASK_BRAND_FILE_PATH, 0)
result = cv2.bitwise_xor(cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY), cv2.bitwise_not(mask_brand))
cv2.imwrite(INPUT_IMAGE_FILE_ONLY_HEARING_AIDS_SKELETON_PATH, result)

cv2.imshow('img', img)
cv2.imshow('original_image', original_image)
cv2.imshow('mask_brand', mask_brand)
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()